In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import time
import numpy as np

In [2]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
    ])
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
    ])
train_set = datasets.ImageFolder(root='data/001/train',transform=train_transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=4, shuffle=True, num_workers=8)

test_set = datasets.ImageFolder(root='data/001/test',transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=4, shuffle=True, num_workers=8)

In [3]:
def train(model, device, criterion, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        loss = criterion(outputs, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {:2d} [{:5d}/{} ({:2.1f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [12]:
def test(model, device, criterion, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += criterion(output, target).item() 
            pred = output.argmax(dim=1)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available(): print(torch.cuda.get_device_name(0))

Tesla K80


In [15]:
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
    
n = len(np.unique(train_set.targets))
classifier = nn.Sequential(nn.Linear(model.fc.in_features,128),
                          nn.ReLU(),
                          nn.Dropout(0.2),
                          nn.Linear(128,64),
                          nn.ReLU(),
                          nn.Dropout(0.2),
                          nn.Linear(64,n))
model.fc = classifier
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

for epoch in range(20):
    train(model, device, criterion, train_loader, optimizer, epoch)
    test(model, device, criterion, test_loader)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/markshi/.torch/models/resnet50-19c8e357.pth
102502400it [00:05, 18118836.91it/s]


Train Epoch:  0 [    0/4654 (0.0%)]	Loss: 2.166622
Train Epoch:  0 [  400/4654 (8.6%)]	Loss: 2.362605
Train Epoch:  0 [  800/4654 (17.2%)]	Loss: 2.050684
Train Epoch:  0 [ 1200/4654 (25.8%)]	Loss: 1.915527
Train Epoch:  0 [ 1600/4654 (34.4%)]	Loss: 1.999390
Train Epoch:  0 [ 2000/4654 (43.0%)]	Loss: 1.520494
Train Epoch:  0 [ 2400/4654 (51.5%)]	Loss: 1.498787
Train Epoch:  0 [ 2800/4654 (60.1%)]	Loss: 2.363690
Train Epoch:  0 [ 3200/4654 (68.7%)]	Loss: 1.520756
Train Epoch:  0 [ 3600/4654 (77.3%)]	Loss: 1.484714
Train Epoch:  0 [ 4000/4654 (85.9%)]	Loss: 1.526124
Train Epoch:  0 [ 4400/4654 (94.5%)]	Loss: 0.914816

Test set: Average loss: 0.6011, Accuracy: 1234/1552 (80%)

Train Epoch:  1 [    0/4654 (0.0%)]	Loss: 1.476590
Train Epoch:  1 [  400/4654 (8.6%)]	Loss: 1.732650
Train Epoch:  1 [  800/4654 (17.2%)]	Loss: 1.758908
Train Epoch:  1 [ 1200/4654 (25.8%)]	Loss: 1.089572
Train Epoch:  1 [ 1600/4654 (34.4%)]	Loss: 0.636830
Train Epoch:  1 [ 2000/4654 (43.0%)]	Loss: 0.471474
Train Ep

In [35]:
test(model, device, criterion, test_loader)


Test set: Average loss: 0.3304, Accuracy: 1397/1552 (90%)



In [47]:
rst = []
with torch.no_grad():
    for i, v in enumerate(test_loader):
        if i==3:
            break
        data, target = v
        data, target = data.to(device), target.to(device)
        output = model(data)
        pred = output.argmax(dim=1)
        print(list(pred.cpu().numpy()))
        rst+=list(pred.cpu().numpy())
rst

[7, 7, 4, 7]
[3, 2, 4, 2]
[7, 1, 4, 0]


[7, 7, 4, 7, 3, 2, 4, 2, 7, 1, 4, 0]

In [48]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F